# Introduction to Polymake in Julia - Exercises

In [1]:
using Polymake
using GAP

polymake version 3.3
Copyright (c) 1997-2019
Ewgenij Gawrilow, Michael Joswig (TU Berlin)
https://polymake.org

This is free software licensed under GPL; see the source for copying conditions.
There is NO warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

Adding path /home/sebastian/Software/GAPJulia/gap/.libs to DL_LOAD_PATH
 ┌───────┐   GAP 4.dev of today
 │  GAP  │   https://www.gap-system.org
 └───────┘   Architecture: x86_64-pc-linux-gnu-julia64-kv6
 Configuration:  gmp 6.1.2, Julia GC, Julia 1.1.0, readline
 Loading the library and packages ...
 Packages:   GAPDoc 1.6.1, PrimGrp 3.3.1, SmallGrp 1.3, TransGrp 2.0.4
 Try '??help' for help. See also '?copyright', '?cite' and '?authors'


In this exercise sheet, we will practise working with Polymake in Julia, and combine it with using GAP. The functions implemented in this exercise sheet are part of the implementation of the GIT fan by Böhm, Keicher, Ren.

## Exercise 1 - Intersecting two cones

Write a function `intersect` that computes the intersection of two (pointed) Polymake cones as a polymake cone, without using the Polymake intersection method.

Hint: You can get the defining inequalities of a Cone `c` via `c.FACETS`

In [2]:
function intersect(p1,p2)
    f1 = p1.FACETS
    f2 = p2.FACETS
    all_ineqs = vcat(f1,f2)
    all_ineqs_jl = convert(Array{Rational{BigInt},2}, all_ineqs)
    return @pm Polytope.Cone(INEQUALITIES=all_ineqs_jl)
end

intersect (generic function with 1 method)

In [3]:
c1 = @pm Polytope.Cone(INPUT_RAYS=[1 0 0; 0 1 0; 0 0 1])
c2 = @pm Polytope.Cone(INPUT_RAYS=[1 0 0; 1 1 0; 1 1 1])
c3 = intersect(c1,c2)
c3.RAYS

polymake: used package cdd
  cddlib
  Implementation of the double description method of Motzkin et al.
  Copyright by Komei Fukuda.
  http://www-oldurls.inf.ethz.ch/personal/fukudak/cdd_home/



pm::Matrix<pm::Rational>
1 1 1
1 0 0
1 1 0


## Exercise 2 - Interior lattice points of polytope

Write a function that, for a given (full dimensional) cone, compute the set of hilbert basis elements that are not rays. You can either use `RAYS` or `INEQUALITIES`

In [4]:
function interior_hilbert_element(cone)
    points = Polytope.hilbert_basis(cone)
    facets = cone.FACETS
    interior_points = []
    for i in 1:size(points)[1]
        current = true
        for j in 1:size(facets)[1]
            if sum(points[i,:] .* facets[j,:]) == 0
                current = false
                break
            end
        end
        if current
            push!(interior_points,points[i,:])
        end
    end
    return convert(Vector{Vector{pm_Integer}},interior_points)
end

interior_hilbert_element (generic function with 1 method)

In [5]:
cone = @pm Polytope.Cone(INPUT_RAYS=[ 1 0 ; 1 5 ])
xx = interior_hilbert_element(cone)

polymake: used package libnormaliz
  [[wiki:external_software#Normaliz]] is a tool for computations in affine monoids, vector configurations, lattice polytopes, and rational cones.
  Copyright by Winfried Bruns, Bogdan Ichim, Christof Soeger.
  http://www.math.uos.de/normaliz/



4-element Array{Array{pm_Integer,1},1}:
 [1, 1]
 [1, 2]
 [1, 3]
 [1, 4]

## Exercise 3 - Operating on cones

Write a function that applies a GAP permutation on $n$ points to a cone of dimension $n$, permuting the coordinates.

Hint: The `@pm` macro matrices currently need to be made from Julia numbers. You can either use `Polymake.perlobj` to create the cone, or convert the integers.

In [6]:
function action(cone,perm)
    rays = cone.RAYS
    new_rays = [ convert(Rational{BigInt},rays[j,i^perm]) for j in 1:size(rays)[1], i in 1:size(rays)[2] ]
    return @pm Polytope.Cone(INPUT_RAYS=new_rays)
end

action (generic function with 1 method)

In [7]:
cone = @pm Polytope.Cone(INPUT_RAYS=[ 1 0 ; 1 5 ])
perm = GAP.EvalString("(1,2)")
action(cone,perm)

type: Cone<Rational>

INPUT_RAYS
0 1
1 1/5

## Exercise 4 - Ordering Cones

For technical reasons, GAP needs to be able to order and compare elements it wants to compute an orbit of.
Write Julia function for `isless` and `==` that provide an arbitrary but fixed order on cones.

In [8]:
function Base.isless(a::Polymake.pm_perl_Object,b::Polymake.pm_perl_Object)
    mat1 = convert(Matrix{Rational{BigInt}},a.RAYS)
    mat2 = convert(Matrix{Rational{BigInt}},b.RAYS)
    return vcat(mat1...) < vcat(mat2...)
end

In [9]:
function Base.:(==)(a::Polymake.pm_perl_Object,b::Polymake.pm_perl_Object)
    mat1 = convert(Matrix{Rational{BigInt}},a.RAYS)
    mat2 = convert(Matrix{Rational{BigInt}},b.RAYS)
    return vcat(mat1...) == vcat(mat2...)
end

In [10]:
cone == cone

true

## Exercise 5 - Orbit computation

Compute the orbit and stabilizer of the cone `c` under the group `g`. What is the size of the orbit.

In [11]:
c = @pm Polytope.Cone(INPUT_RAYS=[1 2 3; 2 4 7; 1 0 3])

type: Cone<Rational>

INPUT_RAYS
1 2 3
1 2 7/2
1 0 3

In [12]:
g = GAP.Globals.SymmetricGroup(3)

GAP: SymmetricGroup( [ 1 .. 3 ] )

In [13]:
orb = GAP.Globals.Orbit(GAP.Globals.SymmetricGroup(3),c,GAP.julia_to_gap(action))

GAP: [ Polymake.pm_perl_ObjectAllocated(Ptr{Nothing} @0x0000000014a4c190), Polymake.pm_perl_ObjectAllocated(Ptr{Nothing} @0x0000000014a9f9f0), Polymake.pm_perl_ObjectAllocated(Ptr{Nothing} @0x0000000014aa77d0), Polymake.pm_perl_ObjectAllocated(Ptr{Nothing} @0x0000000014aa8ef0), Polymake.pm_perl_ObjectAllocated(Ptr{Nothing} @0x0000000014ac8390), Polymake.pm_perl_ObjectAllocated(Ptr{Nothing} @0x0000000014ac1190) ]

In [14]:
length(orb)

6